## Importing essential modules

In [ ]:
import numpy as np
from numpy.polynomial.hermite import hermfit, hermval, hermval2d, hermgrid2d
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import imageio
import os
from tqdm import tqdm
import time
import cv2

from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.filters import gaussian
from sklearn.decomposition import PCA

%matplotlib inline

## Modules

In [ ]:
img1 = np.zeros((7, 7))
img1[1, 4] = 2.5
img1[5, 4] = 2.5
img1[1, 2] = 2.5
img1

In [ ]:
peaks = peak_local_max(img1, min_distance=2, exclude_border=False)

## To avoid multiple peaks in one filter window (replaced by their avg instead)

In [ ]:
def one_peak_per_filter(peaks, separation):
    Updated_peaks = []
    i = 0
    while len(peaks>0):
        i_pick = np.where((peaks[:,0] >= peaks[i,0]-separation) & (peaks[:,0] <= peaks[i,0]+separation) & \
                          (peaks[:,1] >= peaks[i,1]-separation) & (peaks[:,1] <= peaks[i,1]+separation))
        peak1 = peaks[i_pick]
        Len = len(peak1)
        if Len>1:
            peak_new = [int(np.sum(peak1[:,0])/Len), int(np.sum(peak1[:,1])/Len)]
        else:
            peak_new = peak1.ravel().tolist()
        Updated_peaks.append(peak_new)
        peaks = np.delete(peaks, i_pick, 0)
        # print(i, 'peak_new: ', peak_new, '\nPeaks: ', peaks)
        i += 1
    return Updated_peaks

## Test on (0,0) mode

In [ ]:
RepoDir = '~/WORK/Beam_auto_alignment/'
TrainingDataFolder = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-11-07_12-09/'
# file = 'Gen_08_time_183_Power_238.00_alignments_0.000000_0.000000_0.000000_0.000000_endMirror_0.000000.png'
# file = 'Gen_09_time_204_Power_180.16_alignments_0.000000_0.000000_0.000000_0.000000_endMirror_0.000000.png'
file = 'Gen_04_time_68_Power_53.14_alignments_0.000000_0.000000_0.000000_0.000000_endMirror_0.000000.png'
# TrainingDataFolder = RepoDir + 'Data/TrainingData/'
# file = 'HG_3_1_0_5435.png'
img = imageio.imread(TrainingDataFolder+file)[43:243, 54:320, 0]
plt.imshow(img, cmap=plt.cm.gray)
plt.colorbar()
mode = Find_mode2(img, separation1=5, corner=0, show_fig=True, show_basis=True)
print(mode)

In [ ]:
# image dimension (n_pixl x n_pixl)
n_pixl=128
include_LG_data=False
RepoDir = '~/WORK/Beam_auto_alignment/'
TrainingDataFolder = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-22_15-29/'
file = 'Gen_23_time_235_Power_927.08_alignments_-0.000301_0.000066_-0.000002_-0.001734_endMirror_0.349677.png'
# TrainingDataFolder = RepoDir + 'Data/TrainingData/'
# file = 'HG_3_1_0_5435.png'
img = imageio.imread(TrainingDataFolder+file)[43:243, 54:320, 0]
plt.imshow(img, cmap=plt.cm.gray)
plt.colorbar()

In [ ]:
mode = Find_mode2(TrainingDataFolder+file, separation1=10, Sigma1=1, Width=10, Show_fig=True, show_peaks=True)
print(mode)

In [ ]:
TrainingDataFolder = RepoDir + 'Data/TrainingData/'
file = 'HG_3_1_0_5435.png'
# file = 'HG_4_4_0_9657.png'
# file = 'HG_3_4_0_61.png'

img = imageio.imread(TrainingDataFolder+file)
# img = 255 - img
img = img[43:243, 54:320]
plt.figure(figsize=(5,5))
plt.imshow(img, cmap=cm.binary_r)
plt.colorbar()
# mean_x = np.sum(np.sum(img, axis=0)*np.arange(128)) / np.sum(img)
# mean_y = np.sum(np.sum(img, axis=1)*np.arange(128)) / np.sum(img)
# plt.plot(mean_x, mean_y, '*')
plt.show()
# print(mean_x, mean_y)
img1 = Thresh(img)
plt.imshow(img1, cmap=cm.binary_r)
plt.colorbar()

In [ ]:
peaks = Find_Peaks(img1, show_ada_thresh=False, show_fig=True)
print(peaks)

In [ ]:
i_corner = peaks[:,0].argmin()
peak_corner = peaks[i_corner]

In [ ]:
d = np.array([((peaks[i,0]-peak_corner[0])**2 + (peaks[i,1]-peak_corner[1])**2) for i in range(len(peaks[:,0]))])
print(d)
d[d==0] = d.max()
print(d)

In [ ]:
i_near1 = d.argmin()
peak_near1 = peaks[i_near1]
print(d)
d[i_near1] = d.max()
i_near2 = d.argmin()
peak_near2 = peaks[i_near2]
print(d)

In [ ]:
basis1, m1 = find_basis(peak_corner, peak_near1)
basis2, m2 = find_basis(peak_corner, peak_near2)

In [ ]:
mode = peaks_to_mode(peak_corner, basis1, basis2, m1, m2, peaks, 10)
assert (mode[0]+1)*(mode[1]+1) == len(peaks), 'Something went wrong! number of peaks \
inferred from mode does not match with actual number of peaks'

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(img)
plt.plot(peaks[:,0], peaks[:,1], 'ko')
# plt.plot([0,pca.components_[0,0]*20], [0,pca.components_[0,1]*20])
# plt.plot([0,pca.components_[1,0]*20], [0,pca.components_[1,1]*20])

# plt.plot([0,basis1[0]*20], [0,basis1[1]*20], 'r')
# plt.plot([0,basis2[0]*20], [0,basis2[1]*20], 'b')

kk = 4
plt.plot([0], [0], 'r*')
plt.plot([peak_corner[0], peak_corner[0]+(peak_near1[0]-peak_corner[0])*kk], 
         [peak_corner[1], peak_corner[1]+(peak_near1[1]-peak_corner[1])*kk], 'r')
plt.plot([peak_corner[0], peak_corner[0]+(peak_near2[0]-peak_corner[0])*kk], 
         [peak_corner[1], peak_corner[1]+(peak_near2[1]-peak_corner[1])*kk], 'y')

plt.show()

The distance of a point projected on the principal axis from origin is given by $d_i = \frac{my_i + x_i}{\sqrt{1+m^2}}$. The distances of these projections on a PA will form $m+1$ clusters.

In [ ]:
def d_projected_PA(PA, Point):
    # slope of PA
    m = PA[1] / PA[0]
    return (m*Point[1] + Point[0]) / np.sqrt(m**2+1)

def find_mode(PA, Points):
    # find PA with lower slope
    m = np.array([abs(PA[0,1] / PA[0,0]), abs(PA[1,1] / PA[1,0])])
    i_PA = np.argmin(m)
    m_min = m.pop(i_PA)
    
    # get projected distances of all points on this axis
    d = []
    for Point in Points:
        d.append(d_projected_PA(PA[i_PA], Point))
    M = find_peaks(d)

In [ ]:
PA, Points = pca.components_, peaks

In [ ]:
m = np.array([abs(PA[0,1] / PA[0,0]), abs(PA[1,1] / PA[1,0])])
i_PA = np.argmin(m)
print(i_PA)

In [ ]:
# get projected distances of all points on this axis
d = []
for Point in Points:
    d.append(d_projected_PA(PA[i_PA], Point))
print(d)

In [ ]:
plt.plot(d, np.ones(len(d)), 'bo')

In [ ]:
dd = find_mode(pca.components_, peaks)
plt.hist(dd)

In [ ]:
X = np.array([[-1, 1], [-2, 1], [-3, 0], [1, 3], [2, 3], [3, 4]])
plt.plot(X[:,0], X[:,1], 'bo')

In [ ]:
pca = PCA(n_components=2)
pca.fit(X)
print('PCA components: ', pca.components_)
print('Variances: ', pca.explained_variance_)

In [ ]:
# def mode_0_n(Num_Peaks):
#     return Num_Peaks - 1

# def mode_m_n(Num_Peaks):
#     return 

# if pca.explained_variance_ratio_.max() >= 0.8:
#     return mode_0_n(num_peaks)
# else:
#     return mode_m_n(num_peaks)

In [ ]:
plt.plot(X[:,0], X[:,1], 'ko')
plt.plot([0,pca.components_[0,0]], [0,pca.components_[0,1]])
plt.plot([0,pca.components_[1,0]], [0,pca.components_[1,1]])
plt.plot([0], [0], 'r*')
plt.show()

## Distance from the first principal axis. 

A line is represented by $Ax + By + C = 0$. Here, we have the principal axis #1 passing through origin with a slope $m = \frac{PC^1_y}{PC^1_x}$. Thus, the line is represented by, $mx - y = 0$.
The distance from an individual point from it is given by $\frac{y_1 - mx_1}{\sqrt{m^2+1^2}}$

In [ ]:
def distance_from_PA(PA, Point):
    m = PA[1] / PA[0]
    return (Point[1] - m*Point[0]) / np.sqrt(m**2+1)

distances_from_PA1 = []
peaks = X
for point in peaks:
    distances_from_PA1.append(distance_from_PA(pca.components_[0], point))

In [ ]:
print(distances_from_PA1)
print(max(distances_from_PA1)-min(distances_from_PA1) - pca.explained_variance_[1]**0.5*4)

In [ ]:
plt.hist(distances_from_PA1)

## Initializations

In [ ]:
# image dimension (n_pixl x n_pixl)
n_pixl=128
include_LG_data=False
RepoDir = '~/WORK/Beam_auto_alignment/'
# TrainingDataFolder = RepoDir + 'Data/TrainingData/'

## Read files

In [ ]:
files = os.listdir(TrainingDataFolder)
for file in files:
    if file[-3:] != 'png':
        files.remove(file)

Labels = []
# We have grayscale images, so while loading the images we will keep color_mode="grayscale"
Images = []
for file in tqdm(files):
    if not include_LG_data:
        if file[:2] == 'LG':
            continue
    # training images
    img = image.load_img(TrainingDataFolder+file, \
                         target_size=(n_pixl,n_pixl,1), color_mode="grayscale")
    img = image.img_to_array(img)
    img = img/255
    Images.append(img)
    # target labels
    Labels.append(file[:6])

## read image data

In [ ]:
# file = 'HG_3_1_0_5435.png'
# file = 'HG_4_4_0_9657.png'
TrainingDataFolder = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-22_15-29/'
file = 'Gen_23_time_235_Power_927.08_alignments_-0.000301_0.000066_-0.000002_-0.001734_endMirror_0.349677.png'
img = imageio.imread(TrainingDataFolder+file)
# img = 255 - img
img = img[43:243, 54:320]
plt.figure(figsize=(10,10))
plt.imshow(img, cmap=cm.binary_r)
plt.colorbar()

## fit an image with mode functions and find the best fit parameters

### thresholding

In [ ]:
plt.imshow(img, cmap=cm.binary_r)
plt.colorbar()
# mean_x = np.sum(np.sum(img, axis=0)*np.arange(128)) / np.sum(img)
# mean_y = np.sum(np.sum(img, axis=1)*np.arange(128)) / np.sum(img)
# plt.plot(mean_x, mean_y, '*')
plt.show()
# print(mean_x, mean_y)
img1 = Thresh(img)
plt.imshow(img1, cmap=cm.binary_r)
plt.colorbar()

# Attempts at finding a method of fitting

In [ ]:
from numpy.polynomial.hermite import hermfit, hermval, hermval2d, hermgrid2d
from numpy.fft import fft2

## Model

In [ ]:
def herm(xvals, center, scale, mode):
    return hermval(2.**0.5*(xvals-center)/scale, mode)

def HG(im_shape, X_center, Y_center, theta, mode_M, mode_N, SCALE, max_mode=10):
    X = np.arange(im_shape[0])
    Y = np.arange(im_shape[1])
    xx, yy = np.meshgrid(X, Y)
    # translation and rotation
    X_new = (xx-X_center)*np.cos(theta) + (yy-Y_center)*np.sin(theta)
    Y_new = -(xx-X_center)*np.sin(theta) + (yy-Y_center)*np.cos(theta)
    # mode coeffs
    C_m = np.zeros(max_mode)
    C_n = np.zeros(max_mode)
    C_m[mode_M] = 1
    C_n[mode_N] = 1
    # Hermite values at each point
    HX = herm(X_new, 0., SCALE, C_m)
    HY = herm(Y_new, 0., SCALE, C_n)
    return (np.exp(-(X_new**2. + Y_new**2.)/SCALE**2.) * HX * HY)**2.

sigma = 15.
a = 64
m, n = (4,0)
IM_shape = (128, 128)
x_c, y_c = (a, a)

img2 = HG(IM_shape, x_c, y_c, np.pi/6., m, n, sigma)
img2 /= img2.max()
plt.imshow(img2, cmap=cm.binary_r, origin='lower')
# plt.colorbar()
plt.show()

In [ ]:
# fft trial
F = abs(fft2(img2))
# fr = np.fft.fftfreq(128)
fr2 = np.fft.fftshift(F)

plt.imshow(fr2, cmap=cm.binary_r)
plt.colorbar()

# Thresholding + Gaussian smoothing + Local Peak collection

In [ ]:
from skimage.feature import peak_local_max
from scipy import ndimage as ndi
from skimage.filters import gaussian

aa = 10
image_max = ndi.maximum_filter(img1[:,:,0], size=aa, mode='constant')
img_g = gaussian(img1[:,:,0],  sigma=1)
coordinates = peak_local_max(img_g, min_distance=aa)

plt.imshow(img1[:,:,0], cmap=plt.cm.gray)
plt.colorbar()
plt.figure()
plt.imshow(image_max)
plt.colorbar()
plt.figure()
plt.imshow(img_g)
plt.colorbar()
# plt.autoscale(False)
plt.plot(coordinates[:, 1],
         coordinates[:, 0], 'r*')
# plt.set_title('Peak local maxima', fontsize=24)

## Resource

Also Check for fitting:
* https://numpy.org/doc/1.15/reference/generated/numpy.polynomial.hermite.hermfit.html
* https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.polynomial.hermite.hermvander2d.html#numpy.polynomial.hermite.hermvander2d

In [ ]:
from skimage import data, io, filters

In [ ]:
image = data.coins()  # or any NumPy array!
edges = filters.sobel(image)
io.imshow(edges)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.filters import threshold_local
from skimage.feature import peak_local_max
from skimage import feature
from skimage.measure import regionprops
import matplotlib.patches as mpatches
from skimage.morphology import label

# Load a small section of the image.
image = data.coins()[0:95, 70:370]

fig, axes = plt.subplots(ncols=2, nrows=3,
                         figsize=(8, 4))
ax0, ax1, ax2, ax3, ax4, ax5  = axes.flat
ax0.imshow(image, cmap=plt.cm.gray)
ax0.set_title('Original', fontsize=24)
ax0.axis('off')

# Since the image is represented by a NumPy array, we can easily perform operations such as building a histogram of the intensity values.

# Histogram.
values, bins = np.histogram(image,
                            bins=np.arange(256))

ax1.plot(bins[:-1], values, lw=2, c='k')
ax1.set_xlim(xmax=256)
ax1.set_yticks([0, 400])
ax1.set_aspect(.2)
ax1.set_title('Histogram', fontsize=24)

# To divide the foreground and background, we threshold the image to produce a binary image. Several threshold algorithms are available. Here, we employ filter.threshold_adaptive where the threshold value is the weighted mean for the local neighborhood of a pixel.

# Apply threshold.

bw = threshold_local(image, 95, offset=-15)

ax2.imshow(bw, cmap=plt.cm.gray)
ax2.set_title('Adaptive threshold', fontsize=24)
ax2.axis('off')

# We can easily detect interesting features, such as local maxima and edges. The function feature.peak_local_max can be used to return the coordinates of local maxima in an image.

# Find maxima.

coordinates = peak_local_max(image, min_distance=20)

ax3.imshow(image, cmap=plt.cm.gray)
ax3.autoscale(False)
ax3.plot(coordinates[:, 1],
         coordinates[:, 0], 'r*')
ax3.set_title('Peak local maxima', fontsize=24)
ax3.axis('off')

# Next, a Canny feature (feature.canny) (Canny, 1986) detects the edge of each coin.

# Detect edges.

edges = feature.canny(image, sigma=3,
                     low_threshold=10,
                     high_threshold=80)

ax4.imshow(edges, cmap=plt.cm.gray)
ax4.set_title('Edges', fontsize=24)
ax4.axis('off')

# Then, we attribute to each coin a label (morphology.label) that can be used to extract a sub-picture. Finally, physical information such as the position, area, eccentricity, perimeter, and moments can be extracted using measure.regionprops.

# Label image regions.

label_image = label(edges)

ax5.imshow(image, cmap=plt.cm.gray)
ax5.set_title('Labeled items', fontsize=24)
ax5.axis('off')

for region in regionprops(label_image):
    # Draw rectangle around segmented coins.
    minr, minc, maxr, maxc = region.bbox
    rect = mpatches.Rectangle((minc, minr),
                              maxc - minc,
                              maxr - minr,
                              fill=False,
                              edgecolor='red',
                              linewidth=2)
    ax5.add_patch(rect)

plt.tight_layout()
plt.show()

# Fitting in Fourier Space

## actual image

In [ ]:
# image dimension (n_pixl x n_pixl)
n_pixl=128
RepoDir = '~/WORK/Beam_auto_alignment/'

# TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-22_15-29/'
# file1 = 'Gen_23_time_235_Power_927.08_alignments_-0.000301_0.000066_-0.000002_-0.001734_endMirror_0.349677.png'
TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-26_13-07/'
file1 = 'Gen_11_time_408_Power_1594.09_alignments_0.000003_0.000019_-0.000030_0.000126_endMirror_0.000004.png'
img1 = imageio.imread(TrainingDataFolder1+file1)[43:243, 54:320, 0]

img1 = gaussian(img1,  sigma=1)
img1 = Thresh(img1)

f1 = np.fft.fft2(img1)
fshift1 = np.fft.fftshift(f1)
magnitude_spectrum1 = np.abs(fshift1)
magnitude_spectrum1 = 20*np.log(magnitude_spectrum1)
magnitude_spectrum1 = np.asarray(magnitude_spectrum1, dtype=np.float32)
# img_and_magnitude1 = np.concatenate((img1, magnitude_spectrum1), axis=1)


plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img1)
plt.subplot(122)
plt.imshow(magnitude_spectrum1)
# plt.colorbar()

In [ ]:
magnitude_spectrum1.max()

## reading simulated image

In [ ]:
# image dimension (n_pixl x n_pixl)
n_pixl=128
RepoDir = '~/WORK/Beam_auto_alignment/'

TrainingDataFolder = RepoDir + 'Data/TrainingData/'
# file = 'HG_3_1_0_5435.png'
file = 'HG_3_0_0_12.png'
img = imageio.imread(TrainingDataFolder+file)
img = 255 - img

f = np.fft.fft2(img)
fshift = np.fft.fftshift(f)
magnitude_spectrum = np.abs(fshift)
# magnitude_spectrum = 20*np.log(magnitude_spectrum)
magnitude_spectrum = np.asarray(magnitude_spectrum, dtype=np.float32)

plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(magnitude_spectrum)
# plt.colorbar()

In [ ]:
np.max(magnitude_spectrum)

In [ ]:
x = np.arange(1000)
y = np.arange(1000)
xx, yy = np.meshgrid(x, y, sparse=True)
z = np.sin((xx/100)**2 + (yy/100)**2) / ((xx/100)**2 + (yy/100)**2)
plt.figure(figsize=(7,7))
h = plt.pcolormesh(x,y,z,cmap=plt.cm.binary_r)
plt.plot([400], [200], 'y*')
plt.figure(figsize=(7,7))
plt.imshow(z,cmap=plt.cm.binary_r)
plt.plot([400], [200], 'y*')
plt.show()

# Image moments and successive rotation fit

In [ ]:
sigma = 2.5e1
a = 100
m, n = (3,0)
IM_shape = (266, 200)
x_c, y_c = (a, a)
theta = 0#np.pi/2.4

img2 = HG(IM_shape, x_c, y_c, theta, m, n, sigma)

img2 = gaussian(img2,  sigma=1)
img2 = Thresh(img2)
# normalized model
img2 /= img2.sum()

Xm, Ym = moment1(img2)
# plt.figure(figsize=(10,10))
Xsigma, Ysigma, s1, s2 = moment2(img2, Xm, Ym, np.tan(theta), 1., show_im=True)

plt.imshow(img2, cmap=plt.cm.binary_r)
plt.colorbar()
plt.plot(Xm, Ym, 'r*')
print(Xsigma, Ysigma, s1, s2)

In [ ]:
def Thresh(IMG1, thresh=0.5):
    IMG = IMG1.copy()
    IMG[IMG < thresh*IMG.max()] = 0.
    IMG[IMG >= thresh*IMG.max()] = 1.
    return IMG

def herm(xvals, center, scale, mode):
    return hermval(2.**0.5 * (xvals-center)/scale, mode)

def HG(im_shape, X_center, Y_center, theta, mode_M, mode_N, SCALE, max_mode=10):
    X = np.arange(im_shape[1])
    Y = np.arange(im_shape[0])
    xx, yy = np.meshgrid(X, Y)
    # translation and rotation
    X_new = (xx-X_center)*np.cos(theta) + (yy-Y_center)*np.sin(theta)
    Y_new = -(xx-X_center)*np.sin(theta) + (yy-Y_center)*np.cos(theta)
    # mode coeffs
    C_m = np.zeros(max_mode)
    C_n = np.zeros(max_mode)
    C_m[mode_M] = 1
    C_n[mode_N] = 1
    # Hermite values at each point
    HX = herm(X_new, 0., SCALE, C_m)
    HY = herm(Y_new, 0., SCALE, C_n)
    return (np.exp(-(X_new**2. + Y_new**2.)/SCALE**2.) * HX * HY)**2.

def d_frm_orig_along_m(XX, YY, M):
    if not (isinstance(XX, float) & isinstance(YY, float)):
        XX, YY = np.meshgrid(XX, YY)
    l = XX**2. + M**2. * YY**2. + 2.*M*XX*YY
    l /= (1+M**2.)
    l = np.round(l, 3) # numerical errors making values negative (of order: 10^-12 or lower)
    l **= 0.5
    return l

def Spread_n_mean(Img, m):
    # distance of all pts from origin along slope
    d_img = d_frm_orig_along_m(np.arange(Img.shape[1]), np.arange(Img.shape[0]), m) # columns are x values and rows are y values
    # extreme points
    Img2 = Img.copy()
    Img2[Img2 > 0] = 1
    imm = (d_img * Img2)
    # plt.imshow(imm)
    pt_max = np.where(imm == imm.max())
    pt_min = np.where(imm == imm[imm > 0.].min())
    # plt.plot(pt_min[1].sum()/len(pt_min[1]), pt_min[0].sum()/len(pt_min[0]), 'r*')
    # plt.plot(pt_max[1].sum()/len(pt_max[1]), pt_max[0].sum()/len(pt_max[0]), 'g*')
    # plt.show()
    pt_max = np.array([pt_max[1].sum()/len(pt_max[1]), pt_max[0].sum()/len(pt_max[0])]) # the first index is row number which is y-axis value
    pt_min = np.array([pt_min[1].sum()/len(pt_min[1]), pt_min[0].sum()/len(pt_min[0])])
    # mean and spread of these points
    mean = (pt_min+pt_max)/2
    spread = (d_img[Img>0].max() - d_img[Img>0].min())/2.
    return mean, spread

def moments(Img1, Slope, scale, show_im=False):
    # moments
    Mean, sprd1 = Spread_n_mean(Img1, Slope)
    if Slope == 0.:
        Slope = 1e-10
    _, sprd2 = Spread_n_mean(Img1, -1./Slope)
    xmean = Mean[0]
    ymean = Mean[1]
    if show_im:
        # scale
        sin = scale*Slope/np.sqrt(1+Slope**2.)
        cos = scale/np.sqrt(1+Slope**2.)
        plt.plot(xmean, ymean, 'r*')
        plt.plot(xmean-sprd1*cos, ymean-sprd1*sin, 'yo')
        plt.plot(xmean+sprd1*cos, ymean+sprd1*sin, 'yo')
        plt.plot(xmean-sprd2*sin, ymean+sprd2*cos, 'go')
        plt.plot(xmean+sprd2*sin, ymean-sprd2*cos, 'go')
    return sprd1, sprd2, xmean, ymean

# def moment1(Img):
#     x = np.arange(Img.shape[1])
#     y = np.arange(Img.shape[0])
#     xmean = np.average(x, weights=Img.sum(axis=0))
#     ymean = np.average(y, weights=Img.sum(axis=1))
#     return xmean, ymean

# def Sigma(Img, Xmean, Ymean, m):
#     # distance of mean pt from origin along slope
#     Dmean = d_frm_orig_along_m(Xmean, Ymean, m)
#     # distance of all pts from origin along slope
#     d_img = d_frm_orig_along_m(np.arange(Img.shape[1]), np.arange(Img.shape[0]), m) # columns are x values and rows are y values
#     d_img -= Dmean
#     # 2nd moment
#     sigma = d_img**2. * Img / Img.sum()
#     sigma = sigma.sum()
#     sigma **= 0.5
#     # max spread
#     spread = (d_img[Img>0].max() - d_img[Img>0].min())/2.
#     return sigma, spread, d_img[Img>0]

# def moment2(Img1, xmean, ymean, Slope, scale, show_im=False):
#     sig1, sprd1, dd1 = Sigma(Img1, xmean, ymean, Slope)
#     if Slope == 0.:
#         Slope = 1e-10
#     sig2, sprd2, dd2 = Sigma(Img1, xmean, ymean, -1./Slope)
#     if show_im:
#         # center
#         plt.plot(xmean, ymean, 'r*')
#         # scale
#         sin = scale*Slope/np.sqrt(1+Slope**2.)
#         cos = scale/np.sqrt(1+Slope**2.)
#         s1 = sprd1
#         s2 = sprd2
#         plt.plot(xmean-s1*cos, ymean-s1*sin, 'yo')
#         plt.plot(xmean+s1*cos, ymean+s1*sin, 'yo')
#         plt.plot(xmean-s2*sin, ymean+s2*cos, 'bo')
#         plt.plot(xmean+s2*sin, ymean-s2*cos, 'bo')
#     return sig1, sig2, sprd1, sprd2

In [ ]:
def get_max_spreads(Img, Thetas1):
    uniq_thetas = np.unique(Thetas1)
    Spreads = np.zeros((max_m+1, max_n+1, Thetas1.shape[2]))
    for Theta in uniq_thetas:
        # indices for Theta
        i_theta = np.where(Thetas1 == Theta)
        spread_im1, spread_im2, _, _ = moments(Img, np.tan(Theta), 1, show_im=False)
        if spread_im1 > spread_im2:
            Spreads[i_theta] = spread_im1
        else:
            Thetas1[i_theta] = Theta+np.pi/2.
            Spreads[i_theta] = spread_im2
    return Thetas1, Spreads

def get_mode_array(X_c, Y_c, spreads2, Thetas2, IM_shape=(200,266)):
    Sim_modes = np.zeros((max_m+1, max_n+1, IM_shape[0], IM_shape[1]))
    for m in range(max_m+1):
        for n in range(min(max_n+1, m+1)):
            Model = HG(IM_shape, X_c, Y_c, Thetas2[m,n], m, n, initial_scale*spreads2[m,n]/mode_spreads[m,n])
            Model = gaussian(Model,  sigma=1)
            Model = Thresh(Model)
            # normalized model
            Model /= Model.sum()
            Sim_modes[m,n,:,:] = Model
    return Sim_modes

def calculate_fit(Im_array, Sim_modes):
    Fit = ((Im_array>0) & (Sim_modes>0)).sum(axis=3).sum(axis=2) - ((Im_array>0) ^ (Sim_modes>0)).sum(axis=3).sum(axis=2)
#     Fit = (Im_array * Sim_modes).sum(axis=3).sum(axis=2) - np.abs(Im_array - Sim_modes).sum(axis=3).sum(axis=2)
    return Fit

def fit_modes(Thetas, Img, x_c, y_c, show_im=False):
    # check spreads along thetas. Change theta to theta+pi/2 if spread2>spread1
    Thetas, spreads = get_max_spreads(Img, Thetas)
    # create image array to match mode array dimentions
    im_array = np.expand_dims(Img, axis=0)
    im_array = np.repeat(im_array, max_m+1, axis=0)
    im_array = np.expand_dims(im_array, axis=1)
    im_array = np.repeat(im_array, max_n+1, axis=1)
    # fit values for each mode for all theta vals
    fit = np.zeros((max_m+1, max_n+1, Thetas.shape[2]))
    for i in range(Thetas.shape[2]):
        # create new mode array using spreads and mean position
        sim_modes = get_mode_array(x_c, y_c, spreads[:,:,i], Thetas[:,:,i])
        # fit
        fit[:,:,i] = calculate_fit(im_array, sim_modes)
    return fit

def pick_best(Fitting_factor, Thetas):
    Best_fit = Fitting_factor[:,:,0]
    Best_theta = Thetas[:,:,0]
    for i in range(Thetas.shape[2]-1):
        i_best = Best_fit < Fitting_factor[:,:,i+1]
        Best_fit[i_best] = Fitting_factor[:,:,i+1][i_best]
        Best_theta[i_best] = Thetas[:,:,i+1][i_best]
    return Best_fit, Best_theta

def find_mode_spreads(Max_M, Max_N, IM_shape=(200,266)):
    Mode_spreads = np.zeros((Max_M+1, Max_N+1))
    for M in range(Max_M+1):
        for N in range(min(Max_N+1, M+1)):
            mode_model = HG(IM_shape, IM_shape[1]/2, IM_shape[0]/2, 0., M, N, initial_scale)
            mode_model = gaussian(mode_model,  sigma=1)
            mode_model = Thresh(mode_model)
            # calculate spread and feed that value
            sprd1, _, _, _ = moments(mode_model, 0., 1, show_im=False)
            Mode_spreads[M,N] = sprd1
    return Mode_spreads

## only dep on spread

In [ ]:
# image dimension (n_pixl x n_pixl)
n_pixl=128
RepoDir = '~/WORK/Beam_auto_alignment/'

# TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-22_15-29/'
# file1 = 'Gen_23_time_235_Power_927.08_alignments_-0.000301_0.000066_-0.000002_-0.001734_endMirror_0.349677.png'
# file1 = 'Gen_11_time_408_Power_1594.09_alignments_0.000003_0.000019_-0.000030_0.000126_endMirror_0.000004.png'
TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-26_13-07/'
file1 = 'Gen_21_time_749_Power_540.16_alignments_0.000000_-0.000000_0.000000_-0.000000_endMirror_-0.000004.png'
img1 = imageio.imread(TrainingDataFolder1+file1)[43:243, 54:320, 0]

img1 = gaussian(img1,  sigma=1)
img1 = Thresh(img1)

s1, s2, Xm, Ym = moments(img1, np.tan(np.pi/13), 1, show_im=True)

print(s1, s2)
plt.imshow(img1, cmap=plt.cm.binary_r)
plt.plot(Xm, Ym, 'r*')
plt.colorbar()

In [ ]:
max_m = 8
max_n = 1
RepoDir = '~/WORK/Beam_auto_alignment/'
no_iters = 4

# scale used to create models and record their spreads for once
initial_scale = 3e1
# find mode spreads
mode_spreads = find_mode_spreads(max_m, max_n)

# TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-22_15-29/'
# file1 = 'Gen_23_time_235_Power_927.08_alignments_-0.000301_0.000066_-0.000002_-0.001734_endMirror_0.349677.png'
TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-26_13-07/'
# file1 = 'Gen_21_time_749_Power_540.16_alignments_0.000000_-0.000000_0.000000_-0.000000_endMirror_-0.000004.png'
file1 = 'Gen_14_time_510_Power_839.02_alignments_-0.000000_0.000019_-0.000000_0.000009_endMirror_0.000000.png'
img1 = imageio.imread(TrainingDataFolder1+file1)[43:243, 54:320, 0]
# Preprocess
img1 = gaussian(img1,  sigma=1)
img1 = Thresh(img1)

img1 /= img1.sum()

# plt.figure()
# plt.imshow(img1, cmap=plt.cm.binary_r)

t1 = time.time()

# get the modes in image
thetas = np.array([0., np.pi/4., np.pi/2.])
delta_theta = thetas[1] - thetas[0]
thetas = np.expand_dims(thetas, axis=0)
thetas = np.repeat(thetas, max_m+1, axis=0)
thetas = np.expand_dims(thetas, axis=1)
thetas = np.repeat(thetas, max_n+1, axis=1)

old_best_fit = np.zeros((max_m+1, max_n+1))
_, _, im_xc, im_yc = moments(img1, np.tan(0.), 1, show_im=False)

for i in range(no_iters):
    delta_theta /= 2.
    if i == 0:
        fitting_factor = fit_modes(thetas, img1, im_xc, im_yc, show_im=False)
        # check the best theta
        best_fit, best_theta = pick_best(fitting_factor, thetas)
    else:
        fitting_factor = fit_modes(thetas, img1, im_xc, im_yc, show_im=False)
        # append the prev best fit data
        fitting_factor = np.append(fitting_factor, np.expand_dims(best_fit, axis=2), axis=2)
        thetas = np.append(thetas, np.expand_dims(best_theta, axis=2), axis=2)
        # check the best theta
        best_fit, best_theta = pick_best(fitting_factor, thetas)
    thetas = np.zeros((best_theta.shape[0], best_theta.shape[1], 2))
    thetas[:,:,0] = best_theta-delta_theta
    thetas[:,:,1] = best_theta+delta_theta

# get best fit mode
i_best = np.where(best_fit == best_fit.max())
print('Theta: ', best_theta[i_best]*180/np.pi, 'Fit: ', best_fit[i_best], 'Mode: ', (i_best[0][0], i_best[1][0]))    

print(time.time() - t1)

In [ ]:
plt.imshow(img1)

# Mode finding with autocorrelation

In [ ]:
Model = HG(img1.shape, im_xc, im_yc, best_theta[i_best][0], i_best[0][0], i_best[1][0], 30)
Model = gaussian(Model,  sigma=1)
Model = Thresh(Model)
plt.imshow(Model)

## Comparison with prev mode finding

In [1]:
cd ..

/home/shreejit/WORK/Beam_auto_alignment/src


In [7]:
from funcs import *
RepoDir = '~/WORK/Beam_auto_alignment/'
TrainingDataFolder1 = RepoDir + 'Data/Actual_cavity_Fittest_points_per_gen_2019-07-26_13-07/'
# file1 = 'Gen_21_time_749_Power_540.16_alignments_0.000000_-0.000000_0.000000_-0.000000_endMirror_-0.000004.png'
file1 = 'Gen_14_time_510_Power_839.02_alignments_-0.000000_0.000019_-0.000000_0.000009_endMirror_0.000000.png'
img1 = imageio.imread(TrainingDataFolder1+file1)[43:243, 54:320, 0]
# Preprocess
img1 = gaussian(img1,  sigma=1)
img1 = Thresh(img1)

img1 /= img1.sum()

In [15]:
t1 = time.time()
for i in range(200):
    mode_new = Find_mode2(img1, separation1=10, corner=0, show_fig=False, show_basis=False)
print(time.time()-t1)

0.3187522888183594
